In [1]:
import pandas as pd
import os

from google.cloud import bigquery
from datetime import datetime, date
import re

pd.set_option('display.max_rows', None)

from pathlib import Path 

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json"
client = bigquery.Client()

now = datetime.now()
day = now.strftime("%d")
month = now.strftime("%m")
year = now.strftime("%Y")

In [3]:
#all mids that are tagged to 2 or more main brands in US
us_dupe_mids = 'WITH dupe_main_mids AS (SELECT mid,COUNT(main_brand_name) AS brand_count FROM (SELECT DISTINCT mid,b.brand_name AS main_brand_name,b2.brand_name AS distribution_brand_name,b3.brand_name AS payment_brand_name FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b.brand_name IS NOT NULL) GROUP BY mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_dist_mids AS (SELECT mid,COUNT(main_brand_name) AS brand_count FROM (SELECT DISTINCT m.mid, b.brand_name AS main_brand_name,b2.brand_name AS distribution_brand_name, b3.brand_name AS payment_brand_name FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b2.brand_name IS NOT NULL) GROUP BY mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_payment_mids AS (SELECT payment_brand_tagged_spend.mid, COUNT(payment_brand_tagged_spend.main_brand_name) AS brand_count FROM (SELECT DISTINCT m.mid, b.brand_name AS main_brand_name, b2.brand_name AS distribution_brand_name, b3.brand_name AS payment_brand_name FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc ON mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.reference.mcc_mapping` mcc ON mcc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm ON mm.merchid = m.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b ON b.brand_id = mm.main_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 ON b2.brand_id = mm.distribution_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 ON b3.brand_id = mm.payment_brand_id WHERE b3.brand_name IS NOT NULL) AS payment_brand_tagged_spend GROUP BY payment_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC) SELECT mid FROM dupe_payment_mids UNION DISTINCT SELECT mid FROM dupe_dist_mids UNION DISTINCT SELECT mid FROM dupe_main_mids'
us_dupe_mids_df = client.query(us_dupe_mids).to_dataframe()

#all mids that are tagged to 2 or more main brands in UK
uk_dupe_mids = 'with dupe_main_mids as (SELECT main_brand_tagged_spend.mid, COUNT(main_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `consumeredgeresearch-169820.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_ats` mm on mm.merchid = m.merchid left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `consumeredgeresearch-169820.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b.brand_name IS NOT NULL) as main_brand_tagged_spend GROUP BY main_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC ), dupe_dist_mids as ( SELECT dist_brand_tagged_spend.mid, COUNT(dist_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `consumeredgeresearch-169820.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `consumeredgeresearch-169820.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b2.brand_name IS NOT NULL) as dist_brand_tagged_spend GROUP BY dist_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC), dupe_payment_mids as ( SELECT payment_brand_tagged_spend.mid, COUNT(payment_brand_tagged_spend.main_brand_name) as brand_count FROM (SELECT distinct m.mid, b.brand_name as main_brand_name, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `consumeredgeresearch-169820.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mm on mm.merchid = m.merchid left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid left join `consumeredgeresearch-169820.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id WHERE b3.brand_name IS NOT NULL) as payment_brand_tagged_spend GROUP BY payment_brand_tagged_spend.mid HAVING brand_count > 1 ORDER BY brand_count DESC) select mid from dupe_payment_mids UNION DISTINCT select mid from dupe_dist_mids UNION DISTINCT select mid from dupe_main_mids'
uk_dupe_mids_df = client.query(uk_dupe_mids).to_dataframe()

In [20]:
#tagged spend based on brand_name variable and spend_amount threshold
brand_name = "'%RAGING%WAVE%'"
spend_amount = '1000'
region = 'US' #'US' or 'UK'

filepath = Path('C:/Users/Greg/OneDrive - Consumer Edge Research LLC/Desktop/Python/Exports/' + brand_name + '_Mid_Check_' + month +'_' + day + '.csv')

In [21]:
if region == 'US':
    brand_tagging = 'SELECT * FROM (SELECT m.mid,LEFT(m.merchdesc,15) as merchdesc,m.mcc,m.state,m.city,m.zip,m.avg_spend_amount,m.spend_amount,m.first_trans_date,m.last_trans_date,m.top_state_abbr,m.pct_state_trans,  main_pattern_match_id,b.brand_name as main_brand_name,b2.brand_name as distribution_brand_name,b3.brand_name as payment_brand_name,mmm.subchannel,m.merchid,m.top_cobrid FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_subchannel_ats` mmm on m.merchid = mmm.merchid) where spend_amount > ' + spend_amount + ' and main_brand_name LIKE ' + brand_name
    brand_tagging_df = client.query(brand_tagging).to_dataframe()

    #unique tagged brand mids
    brand_mids = brand_tagging_df['mid'].drop_duplicates()
    
    #set difference of unique tagged brand mids and known shared US mids
    unique_brand_mids = list(set(brand_mids) - set(us_dupe_mids_df['mid']))

    mid_sql = '''SELECT * FROM (SELECT  b.brand_name as main_brand_name, m.mid, m.merchdesc, m.mcc, mc.mcc_category, m.city, m.state, m.zip, m.avg_spend_amount, m.spend_amount, m.first_trans_date, m.last_trans_date, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name, m.merchid FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where main_brand_name IS NULL and mid IN UNNEST(%s)''' %(unique_brand_mids)
    mid_sql = mid_sql + ' and spend_amount > ' + spend_amount + ' ORDER BY spend_amount DESC'
    untagged_spend = client.query(mid_sql).to_dataframe()

if region == 'UK':
    brand_tagging = 'SELECT m.merchid,m.mid,m.merchdesc,m.mcc,m.location,m.postcode,round(m.avg_spend_amount, 0) as avg_spend_amount,round(m.stddev_spend_amount, 0) as stddev_spend_amount,round(m.spend_amount, 0) as spend_amount,m.card_count,m.trans_count,mm.channel,m.first_trans_date,m.last_trans_date,m.day_count,m.top_cardtype,b.brand_name as main_brand_name,b2.brand_name as distribution_brand_name,b3.brand_name as payment_brand_name,kmm.brand_source,b4.brand_name as kmm_main_brand_name,mmm.main_pattern_match_id FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid LEFT JOIN `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on m.merchid = mmm.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id LEFT JOIN `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match` kmm on m.merchid = kmm.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b4 on b4.brand_id = kmm.main_brand_id where spend_amount > ' + spend_amount + ' and b.brand_name LIKE ' + brand_name
    brand_tagging_df = client.query(brand_tagging).to_dataframe()

    #unique tagged brand mids
    brand_mids = brand_tagging_df['mid'].drop_duplicates()
    
    #set difference of unique tagged brand mids and known shared UK mids
    unique_brand_mids = list(set(brand_mids) - set(uk_dupe_mids_df['mid']))

    mid_sql = '''SELECT DISTINCT m.mid, m.merchdesc, m.mcc, mcc.mcc_category, round(m.spend_amount, 0) as spend_amount, FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m LEFT JOIN `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_ats` mm on m.merchid = mm.merchid left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on mmm.merchid = m.merchid left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `consumeredgeresearch-169820.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `consumeredgeresearch-169820.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id where b.brand_name IS NULL and mid IN UNNEST(%s)'''%(unique_brand_mids)
    mid_sql = mid_sql + ' and spend_amount > ' + spend_amount + ' ORDER BY spend_amount DESC'
    untagged_spend = client.query(mid_sql).to_dataframe()

In [22]:
untagged_spend['mid'] = untagged_spend['mid'].str.lstrip("0")
untagged_spend_mids = untagged_spend['mid'].drop_duplicates()

mid_regex = '^0*('
for i, line in enumerate(untagged_spend_mids):
    if i < len(untagged_spend_mids)-1:
        regex_i = line + '|'
        mid_regex += regex_i
    else:
        regex_i = line + ')$'
        mid_regex += regex_i
mid_regex

'^0*('

In [23]:
if region == 'US':
    sql_qa = "SELECT mid, merchdesc, mcc, mcc_category, spend_amount, first_trans_date, main_brand_name FROM (SELECT  b.brand_name as main_brand_name, m.mid, m.merchdesc, m.mcc, mc.mcc_category, m.city, m.state, m.zip, m.avg_spend_amount, m.spend_amount, m.first_trans_date, m.last_trans_date, b2.brand_name as distribution_brand_name, b3.brand_name as payment_brand_name, m.merchid FROM `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `consumeredgeresearch-169820.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where regexp_contains(mid, '" + mid_regex + "') and spend_amount > " + spend_amount + " and main_brand_name IS NULL ORDER BY spend_amount DESC"
    sql_qa_df = client.query(sql_qa).to_dataframe()
    
if region == 'UK':
    sql_qa = "SELECT DISTINCT m.mid, m.merchdesc, m.mcc, mcc.mcc_category, round(m.spend_amount, 0) as spend_amount, round(m.avg_spend_amount, 0) as avg_spend_amount, b.brand_name FROM `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc left join `consumeredgeresearch-169820.reference.mcc_mapping` mcc on mcc.mcc = m.mcc left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_ats` mm on mm.merchid = m.merchid left join `consumeredgeresearch-169820.apollo_uk_ats_ml.mid_match_brand_ats` mmm on mmm.merchid = m.merchid left join `consumeredgeresearch-169820.ground_truth.brand_detail` b on b.brand_id = mm.main_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b2 on b2.brand_id = mm.distribution_brand_id left join `consumeredgeresearch-169820.ground_truth.brand_detail` b3 on b3.brand_id = mm.payment_brand_id where regexp_contains(mid, '" + mid_regex + "') and spend_amount > " + spend_amount + " ORDER BY spend_amount DESC"
    sql_qa_df = client.query(sql_qa).to_dataframe()

BadRequest: 400 Cannot parse regular expression: missing ): ^0*(; reason: invalidQuery, location: query, message: Cannot parse regular expression: missing ): ^0*(

Location: US
Job ID: c53bcfe4-2415-4a1f-aef4-a4a6dfd9ebe7


In [ ]:
#Untagged merchdesc using shared mids
sql_qa_df.head(200)

,mid,merchdesc,mcc,mcc_category,spend_amount,first_trans_date,main_brand_name
0,580306301443630,BIG KAHUNA WEB,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",145533.310000,2015-04-17,None
1,984250705881,TIL*CE BIG KAHUNA' 000-000-0000,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",110639.310000,2023-02-16,None
2,984250705881,TIL*CE BIG KAHUNA',7996,"Amusement Parks, Circuses, Carnivals, and Fort...",93390.600000,2023-01-24,None
3,518993321118986,BIG KAHUNAS WATERP 850-837-8319,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",88410.900000,2023-05-05,None
4,580130701443630,BIG KAHUNA,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",73461.860000,2015-03-30,None
5,518993321118986,BIG KAHUNAS WATERP,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",63659.670000,2022-07-13,None
6,518993321118986,BIG KAHUNAS WATERPARK,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",37713.580000,2023-05-06,None
7,580306301443630,BIG KAHUNA WEB,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",30197.890000,2015-04-16,None
8,518993321118986,BIG KAHUNAS WATERP,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",27596.240000,2022-06-22,None
9,000984250705881,TIL*CE BIG KAHUNA'S ON,7996,"Amusement Parks, Circuses, Carnivals, and Fort...",21899.940000,2023-02-04,None


In [ ]:
mcc = '(' + "'5812'," + "'7999'" + ')'

brand_merchdesc = brand_tagging_df['merchdesc'].drop_duplicates().dropna()
unique_brand_merchdesc = list(brand_merchdesc)

merchdesc_sql = '''SELECT * FROM (SELECT  b.brand_name as main_brand_name, m.merchdesc, m.mcc, mc.mcc_category,m.avg_spend_amount, m.spend_amount FROM `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_enhanced` m left join `consumeredgeresearch-169820.apollo_panel_map.mcc_category` mc on mc.mcc = m.mcc LEFT JOIN `beta-omega-alpha-theta.apollo_us_combined_ats_ml.mid_match_brand_ats` mm on m.merchid = mm.merchid LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b on mm.main_brand_id = b.brand_id  LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b2 on mm.distribution_brand_id = b2.brand_id LEFT JOIN `beta-omega-alpha-theta.ground_truth.brand_detail` b3 on mm.payment_brand_id = b3.brand_id) where main_brand_name IS NULL and merchdesc IN UNNEST(%s)''' %(unique_brand_merchdesc)
merchdesc_sql += ' and spend_amount > ' + spend_amount + ' and mcc IN ' + mcc + ' ORDER BY spend_amount DESC'

merchdesc_df = client.query(merchdesc_sql).to_dataframe()

In [ ]:
#Similar untagged merchdesc (Will commonly be due to maroon missing mids pre 2022)
merchdesc_df

,main_brand_name,merchdesc,mcc,mcc_category,avg_spend_amount,spend_amount
0,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,41.568630,436470.610013
1,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,48.142911,312062.350072
2,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,46.786451,304907.300000
3,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,35.681599,246631.210000
4,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,42.479065,152627.280000
5,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,50.210576,116890.219994
6,None,BIG KAHUNA,5812,Eating Places and Restaurants,31.712689,114387.670000
7,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,38.128507,103938.310000
8,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,45.916881,67268.229971
9,None,TST* BIG KAHUNA,5812,Eating Places and Restaurants,45.535541,61882.799986
